In [ ]:
import pyautogui as pg

tgtregion = (1322, 679, 50, 150)
pg.screenshot(region=tgtregion)



In [ ]:
import pytesseract
import pyautogui as pg
import numpy as np 
import cv2 
import re
import time
import matplotlib.pyplot as plt
import beepy
from tsapi.utils import save_as_yaml

def alert_con_100(reslist, lookbackw=20*60//5):
    con0 = np.max(reslist[-lookbackw:])
    con1 = np.min(reslist[-(lookbackw + np.argmax(reslist[-lookbackw:])):])
    
    if con0>100 and con1>70:
        beepy.beep(1)
        

def check_posneg(rois):
    # Optional: preprocess ROI (convert to grayscale, threshold)
    def _preprocess(roi):
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)  # Enlarge
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        thresh = cv2.bitwise_not(thresh)
        return thresh

    for i, roi in enumerate(rois):
        processed = _preprocess(roi)
        text = pytesseract.image_to_string(processed, config='--psm 7 -c tessedit_char_whitelist=-0123456789')
#         text = pytesseract.image_to_string(processed, config='--psm 7 digits')
#         xnumber = re.findall("[-]+[0-9]+[0-9.]+[0-9]", text.strip())[0]
#         if '-' in xnumber:
            
        print(f"Box check_posneg {i+1}: {text.strip()},")
        
    return text.strip()

def identify_number(tgtregion = (1322, 679, 50, 150), slp=5, reslist=[]):
    img = np.array(pg.screenshot(region=tgtregion))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    red_lower = np.array([136, 87, 111], np.uint8) 
    red_upper = np.array([180, 255, 255], np.uint8) 
    red_mask = cv2.inRange(hsv, red_lower, red_upper)
    
    plt.imshow(red_mask, cmap='gray')
    plt.show()
    
    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Optional: sort contours by area if needed
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    rois = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        roi = img[y:y+h, x:x+w]
        rois.append(roi)

    import pytesseract

    # Optional: preprocess ROI (convert to grayscale, threshold)
    def preprocess(roi):
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return thresh

    for i, roi in enumerate(rois):
        processed = preprocess(roi)
        text = pytesseract.image_to_string(processed, config='--psm 7 digits')
        print(f"Box identify_number {i+1}: {text.strip()}")           
        xnumber = re.findall("[0-9]+[0-9.]+[0-9]", text.strip())[0]
        
        
    
    
    try:
        posneg = check_posneg(rois)
        
        res = float(xnumber)
        if res>130 or "." not in xnumber:
            if len(posneg.replace("-", ""))==4:
                res = float(posneg.replace("-", "")) / 100
            
        if "-" in posneg:
            res *= -1
        
        print(res)
        reslist.append(res)
        
        alert_con_100(reslist, lookbackw=5*60//slp)
        
        save_as_yaml("reslist.yaml", reslist)
    except Exception as e:
        print(e)
    
    time.sleep(slp)
    
    return reslist

In [ ]:
reslist = []
while True:
    try:
        reslist = identify_number(tgtregion = (1322, 679, 50, 150), reslist=reslist)
    except:
        pass